In [1]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np

In [14]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=1.5))
mesh.Curve(3); Draw(mesh)
# viscosity
nu = 0.001
t = 0
tend = 0
dt = 0.01
dn = 1/dt

In [15]:
k = 3
V = VectorH1(mesh,order=k, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q])

In [16]:
gfu = GridFunction(X)
velocity = gfu.components[0]
Draw(velocity,mesh,"u",sd=3)
Draw(gfu.components[1],mesh,"p",sd=3)
from ngsolve.internal import visoptions
visoptions.scalfunction = "u:0"

# parabolic inflow at bc=1:
uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

Redraw()



In [17]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

rows,cols,vals = a.mat.COO()
A_np = sp.csr_matrix((vals,(rows,cols)))

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

Redraw()



In [18]:
conv = BilinearForm(X)
conv += grad(velocity)*velocity*v*dx



In [19]:
# implicit Euler/explicit Euler splitting method:
from scipy.sparse.linalg import inv
MA = BilinearForm(X)
MA += InnerProduct(u,v)*dx + dt*stokes
MA.Assemble()
rows,cols,vals = MA.mat.COO()
MA_np = sp.csr_matrix((vals,(rows,cols)))

for i in range(int(dn)):
    print ("\rt=", t, end="" )
    conv.Assemble()
    rows,cols,vals = conv.mat.COO()
    Conv_np = sp.csr_matrix((vals,(rows,cols)))

    Mstar_np = MA_np + dt * Conv_np
    #print(Mstar_np[0,0])
    #print(type(Mstar_np))
    inv_Mstar_np = inv(Mstar_np)
    res = a.mat * gfu.vec + conv.mat * gfu.vec
    gfu_np = np.array(gfu.vec)
    res_np = A_np.dot(gfu_np) + Conv_np.dot(gfu_np)
    gfu_np -= dt * inv_Mstar_np.dot(res_np)
    gfu.vec.FV().NumPy()[:] = gfu_np











    #MA += InnerProduct(u,v)*dx + dt*stokes# + dt*c
    #MA.Assemble()#

    #conv.Assemble()
    #Mstar = MA.mat + conv.mat
    #Mstar.Assemble()
    #conv += dt*InnerProduct(grad(velocity)*velocity,v)*dx
    #print(conv.mat)
    #conv.Assemble()
    #Mstar = mNS + conv.mat
    #inv = Mstar.Inverse(X.FreeDofs())
    #rows,cols,vals = Mstar.COO()
    #inv_np = sp.csr_matrix((vals,(rows,cols)))

    #res = a.mat * gfu.vec + conv.mat * gfu.vec
    
    #res_np = np.array(res)
    #gfu_np = np.array(gfu.vec)
    #print(type(gfu_np))
    #print(type(res_np))
    #print(type(inv))
    #gfu_np = gfu_np - inv.dot(res_np)

    t = t + dt
    Redraw()

t= 0.09999999999999999

KeyboardInterrupt: 